### Abdullah Nasih Jasir (Абдулла Насих Джасир)
---

# **BIOLOGICAL RESPONSE**

This project is used to predict activity field of Biological Response of The Molecul. I use four different methods
1. Small Decision Tree
2. Deep Decision Tree
3. Random Forest on Small Trees
4. Random Forest on Deep Trees
---

In [1]:
import pandas as pd

# Data Wrangling